In [2]:
import sqlite3
import random
from datetime import datetime, timedelta

# 设置随机数生成器的种子
random.seed(42)

# 连接到SQLite数据库（如果不存在则创建）
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

# 创建表
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales_purchase (
    company_name TEXT,
    customer_name TEXT,
    sales_amount FLOAT,
    purchase_amount FLOAT,
    gross_profit FLOAT,
    gross_margin FLOAT,
    sales_settlement_date DATE,
    purchase_settlement_date DATE,
    product_category TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS coal_inventory (
    warehouse_name TEXT,
    product_type TEXT,
    inventory_weight FLOAT,
    estimated_inventory_value FLOAT,
    advance_payment FLOAT,
    update_date DATE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS steel_inventory (
    warehouse_name TEXT,
    product_type TEXT,
    inventory_quantity FLOAT,
    inventory_value FLOAT,
    update_date DATE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS agency_business (
    company_name TEXT,
    customer_name TEXT,
    product_category TEXT,
    customer_inventory_value FLOAT,
    update_date DATE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS receivables_business (
    company_name TEXT,
    customer_name TEXT,
    receivables_balance FLOAT,
    update_date DATE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS ageing_business (
    company_name TEXT,
    customer_name TEXT,
    account_amount FLOAT,
    ageing TEXT,
    is_anomaly BOOLEAN
)
''')

# 公司简称到全称的映射
company_short_names = {
    "湖北国贸能源化工有限公司": "国贸能化",
    "湖北国贸金属矿产有限公司": "国贸金属矿",
    "湖北国贸汽车有限公司": "国贸汽车",
    "湖北国际贸易集团有限公司": "国贸集团",
    "湖北国贸农产品有限公司": "国贸农产品",
    "武汉鼎联丰国际贸易有限公司": "鼎联丰",
    "湖北国贸农产品有限公司武汉分公司": "国贸农产武汉",
    "湖北南方大集实业有限公司": "南方大集",
    "湖北南方大集实业有限公司东西湖分公司": "南方大集东西湖",
    "湖北南方大集实业有限公司慈惠分公司": "南方大集慈惠",
    "湖北南方大集实业有限公司江汉分公司": "南方大集江汉",
    "湖北南方大集实业有限公司能源分公司": "南方大集能源",
    "湖北南方工贸有限公司": "南方工贸",
    "湖北南方集团有限公司": "南方集团",
    "湖北国贸供应链管理有限公司": "国贸供应链",
    "湖北华中能源发展有限公司": "华中能源",
    "湖北国贸汽车有限公司红安分公司": "国贸汽车红安",
}
# 生成随机数据
def generate_random_data():
    customer_names = ["客户A", "客户B", "客户C", "客户D"]
    product_categories = ["煤炭", "钢材", "农产品", "汽车"]
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2024, 12, 31)
    current_date = start_date

    # 仓库名称列表
    warehouses = [f"仓库{i}" for i in range(1, 21)]  # 生成20个仓库名称

    # 生成采购和销售数据
    while current_date <= end_date:
        for company in company_short_names.values():
            # 生成采购数据
            purchase_amount = round(random.uniform(8000, 95000), 2)
            purchase_settlement_date = current_date.strftime('%Y-%m-%d')
            product_category = random.choice(product_categories)

            cursor.execute('''
            INSERT INTO sales_purchase (company_name, customer_name, sales_amount, purchase_amount, gross_profit, gross_margin, sales_settlement_date, purchase_settlement_date, product_category)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (company, random.choice(customer_names), None, purchase_amount, None, None, None, purchase_settlement_date, product_category))

            # 生成销售数据
            sales_amount = round(random.uniform(10000, 100000), 2)
            gross_profit = sales_amount - purchase_amount
            gross_margin = gross_profit / sales_amount if sales_amount > 0 else 0
            sales_settlement_date = current_date.strftime('%Y-%m-%d')

            cursor.execute('''
            UPDATE sales_purchase
            SET sales_amount = ?, gross_profit = ?, gross_margin = ?, sales_settlement_date = ?
            WHERE company_name = ? AND purchase_settlement_date = ? AND product_category = ?
            ''', (sales_amount, gross_profit, gross_margin, sales_settlement_date, company, purchase_settlement_date, product_category))

        current_date += timedelta(days=30)  # 每个月添加一次

    # 修改表结构，添加 company_name 列
    cursor.execute('''
    ALTER TABLE coal_inventory ADD COLUMN company_name TEXT
    ''')

    cursor.execute('''
    ALTER TABLE steel_inventory ADD COLUMN company_name TEXT
    ''')

    # 生成煤炭库存数据时添加公司名称
    for _ in range(100):  # 生成10条煤炭库存数据
        company_name = random.choice(list(company_short_names.values()))  # 随机选择公司名称
        warehouse_name = random.choice(warehouses)
        product_type = "煤炭"
        inventory_weight = round(random.uniform(5000, 20000), 2)
        estimated_inventory_value = round(inventory_weight * random.uniform(300, 500), 2)
        advance_payment = round(random.uniform(100000, 500000), 2)
        update_date = datetime.now().strftime('%Y-%m-%d')
        
        cursor.execute('''
        INSERT INTO coal_inventory (company_name, warehouse_name, product_type, inventory_weight, estimated_inventory_value, advance_payment, update_date)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (company_name, warehouse_name, product_type, inventory_weight, estimated_inventory_value, advance_payment, update_date))

    # 生成钢材库存数据时添加公司名称
    for _ in range(100):  # 生成10条钢材库存数据
        company_name = random.choice(list(company_short_names.values()))  # 随机选择公司名称
        warehouse_name = random.choice(warehouses)
        product_type = "钢材"
        inventory_quantity = round(random.uniform(1000, 10000), 2)
        inventory_value = round(inventory_quantity * random.uniform(1000, 2000), 2)
        update_date = datetime.now().strftime('%Y-%m-%d')
        
        cursor.execute('''
        INSERT INTO steel_inventory (company_name, warehouse_name, product_type, inventory_quantity, inventory_value, update_date)
        VALUES (?, ?, ?, ?, ?, ?)
        ''', (company_name, warehouse_name, product_type, inventory_quantity, inventory_value, update_date))

    # 生成代理业务数据
    for _ in range(100):  # 生成10条代理业务数据
        company_name = random.choice(list(company_short_names.values()))
        customer_name = random.choice(customer_names)
        product_category = random.choice(product_categories)
        customer_inventory_value = round(random.uniform(100000, 1000000), 2)
        update_date = datetime.now().strftime('%Y-%m-%d')
        
        cursor.execute('''
        INSERT INTO agency_business (company_name, customer_name, product_category, customer_inventory_value, update_date)
        VALUES (?, ?, ?, ?, ?)
        ''', (company_name, customer_name, product_category, customer_inventory_value, update_date))

    # 生成应收账款数据
    for _ in range(100):  # 生成10条应收账款数据
        company_name = random.choice(list(company_short_names.values()))
        customer_name = random.choice(customer_names)
        receivables_balance = round(random.uniform(50000, 500000), 2)
        update_date = datetime.now().strftime('%Y-%m-%d')
        
        cursor.execute('''
        INSERT INTO receivables_business (company_name, customer_name, receivables_balance, update_date)
        VALUES (?, ?, ?, ?)
        ''', (company_name, customer_name, receivables_balance, update_date))

    # 生成账龄数据
    for _ in range(100):  # 生成10条账龄数据
        company_name = random.choice(list(company_short_names.values()))
        customer_name = random.choice(customer_names)
        account_amount = round(random.uniform(100000, 1000000), 2)
        ageing = random.choice(["1-30天", "31-60天", "61-90天", "91天以上"])
        is_anomaly = random.choice([True, False])
        
        cursor.execute('''
        INSERT INTO ageing_business (company_name, customer_name, account_amount, ageing, is_anomaly)
        VALUES (?, ?, ?, ?, ?)
        ''', (company_name, customer_name, account_amount, ageing, is_anomaly))

# 执行生成数据
generate_random_data()

# 提交事务
conn.commit()

# 关闭连接
conn.close()

print("Random data generated and inserted into the database with a fixed seed for reproducibility.")

Random data generated and inserted into the database with a fixed seed for reproducibility.
